In [98]:
%load_ext autoreload
%autoreload 2

In [99]:
from refinegems.classes.gapfill import KEGGapFiller
from refinegems.utility.io import load_model

In [42]:
mpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435.xml'

model = load_model(mpath,'libsbml')

In [98]:
gff2 = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435_RefSeq.gff'
missing_genes = get_missing_genes(gff2,load_model('/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435.xml','libsbml'))
missing_genes

# @TODO : some od the genes do not have an old locus tag -> what to do?

,ncbiprotein,locus_tag,ec-code
0,[WP_011274359.1],SH0001,None
1,[WP_011274360.1],SH0002,None
2,[WP_172458781.1],SH0003,None
3,[WP_011274362.1],SH0004,None
4,[WP_011274363.1],SH0005,None
...,...,...,...
2573,[WP_145424740.1],pSHaeC05,None
2574,[WP_011276926.1],pSHaeC06,None
2575,[WP_011276927.1],pSHaeC07,None
2576,[WP_011276928.1],pSHaeC08,None


In [129]:
from refinegems.classes.gapfill import GeneGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

gffpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_RefSeq.gff'

gf2 = GeneGapFiller()
gf2_missing_genes = gf2.get_missing_genes(gffpath,model)
# ncbiprotein | locus_tag | ec-code

In [130]:
gf2_missing_genes

,ncbiprotein,locus_tag,ec-code
0,WP_011274359.1,SH0001,None
1,WP_011274360.1,SH0002,None
2,WP_172458781.1,SH0003,None
3,WP_011274362.1,SH0004,None
4,WP_011274363.1,SH0005,None
...,...,...,...
2573,WP_145424740.1,pSHaeC05,None
2574,WP_011276926.1,pSHaeC06,None
2575,WP_011276927.1,pSHaeC07,None
2576,WP_011276928.1,pSHaeC08,None


In [120]:
import pandas as pd
from refinegems.classes.gapfill import map_ec_to_reac
from refinegems.curation.db_access.db import get_ec_from_ncbi
# for notebook, later should be part of object
manual_curation = {}

# find missing reaction
def get_missing_reacs(missing_genes, mail:str=None, check_NCBI:bool=False):
    
    # Case 1:  no EC, no ncbiprotein
    # ------------------------------
    case_1 = missing_genes[missing_genes['ncbiprotein'].isna() & missing_genes['ec-code'].isna()]
    if len(case_1) > 0:
        pass
#       -> BLAST against SwissProt to get EC/BRENDA 
#          (mappings + sequences can be downloaded from DB)
            # two tables: 
            #   1.: TSV with UniprotID, BRENDA and EC -7.7MB (26.07.2024)
            #   2.: FASTA with sequences ~280MB (26.07.2024)
            # more infos:
            #   - API generation possible for automatic download
            #   - UniprotID can be extracted from FASTA header sp|A0A1B0GTW7|...
            #   - maybe write a small script to download stuff and shorten to actual EC/BRENDA mappings
#       -> sth like DeepECTransformer (tools either not good, 
#          not installable or no license)
#       -> use ECRECer Web service output as input 
#          (whole protein fasta -> wait for Gwendolyn's results) -> does not work / not return
#       -> use CLEAN webservice

    manual_curation['no ncbiprotein, no EC'] = case_1[case_1['ncbiprotein'].isna() & case_1['ec-code'].isna()] 
    mapped_reacs = pd.concat(case_1[~(case_1['ncbiprotein'].isna() & case_1['ec-code'].isna())] , 
                missing_genes[~(missing_genes['ncbiprotein'].isna() & missing_genes['ec-code'].isna())])

    # Case 2: no EC but ncbiprotein
    # ----------------------------- 
    #       -> access ncbi for ec (optional) 
    if check_NCBI and mail:
        mapped_reacs['ec-code'] = mapped_reacs.progress_apply(lambda x: get_ec_from_ncbi(mail,x['ncbiprotein']) if not x['ec-code'] else x['ec-code'], axis=1)
    
    manual_curation['no EC'] = mapped_reacs[mapped_reacs['ec-code'].isna()]
    mapped_reacs = mapped_reacs[~mapped_reacs['ec-code'].isna()]

    # Case 3: already has EC
    # ----------------------
    #       -> directly access reaction 
    mapped_reacs = map_ec_to_reac(mapped_reacs[['ec-code','ncbiprotein']])

NameError: name 'map_ec_to_reac' is not defined

In [173]:
from Bio import SeqIO
from pathlib import Path

outdir = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/'
fasta = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_proteins_genome.fasta'


def create_missing_genes_protein_fasta(fasta,outdir, missing_genes):
    
    # format the missing genes' locus tags
    locus_tags_list = list(missing_genes['locus_tag'])
    locus_tags_dict = {_:'[locus_tag='+_+']' for _ in list(missing_genes['locus_tag'])}
    
    # parse the protein FASTA
    protfasta = SeqIO.parse(fasta,'fasta')
    
    # extract the sequences of the missing genes only
    missing_seqs = []
    for seq in protfasta:
        # Case 1: locus tag equals FASTA header
        if seq.id in locus_tags_dict.keys():
            missing_seqs.append(seq)
        # Case 2: locus tag as descriptor
        elif any([k for k,v in locus_tags_dict.items() if v in seq.description]):
            for k,v in locus_tags_dict.items():
                if v in seq.description:
                    seq.id = k
                    missing_seqs.append(seq)
                    break
        # Case _: locus tag either in model or errounous
        else:
            pass
        
    # save the collected sequences in a new file
    outfile = Path(outdir,'missing_genes.fasta')
    SeqIO.write(missing_seqs, outfile, 'fasta')
    
    return outfile


create_missing_genes_protein_fasta(fasta,outdir,gf2_missing_genes)

PosixPath('/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/missing_genes.fasta')

In [191]:
import subprocess
from typing import Literal

def run_DIAMOND_blastp(fasta:str, db:str, 
                       sensitivity:Literal['sensitive', 'more-sensitive', 'very-sensitive','ultra-sensitive']='more-sensitive',
                       coverage:float=95.0,
                       threads:int=2,
                       outdir:str=None, outname:str='DIAMOND_blastp_res.tsv'):
    
    if outdir:
        outname = Path(outdir,'DIAMOND_blastp_res.tsv')
        logfile = Path(outdir,'log_DIAMOND_blastp.txt')
    else:
        outname = Path(outname)
        logfile = Path('log_DIAMOND_blastp.txt')
      
    # @TODO: test, if it works with different paths and their problems  
    # @TODO: write additional output to a logfile, not stderr
    subprocess.run([F'diamond blastp -d {db} -q {fasta} --{sensitivity} --query-cover {coverage} -p {int(threads)} -o {outname} --outfmt 6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore > {logfile}'], shell=True)

    return outname

In [195]:
test_fasta = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/mini_test.fasta'
db = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/swissprot.dmnd'
outdir = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/'

run_DIAMOND_blastp(test_fasta, db, outdir=outdir)

diamond v2.1.9.163 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 2
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: /Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435
#Target sequences to report alignments for: 25
Opening the database...  [0.017s]
Database: /Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/swissprot.dmnd (type: Diamond database, sequences: 571864, letters: 207016062)
Block size = 2000000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0s]
Masking queries...  [0s]
Algorithm: Double-indexed
Building query histograms...  [0s]
Seeking in database...  [0s]
Loading reference sequences...  [0.238s]
Masking reference...  [9.99s

PosixPath('/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/DIAMOND_blastp_res.tsv')

In [206]:
def filter_DIAMOND_blastp_results(blasttsv:str, pid_theshold:float=90.0):
    
    if pid_theshold > 100.0 or pid_theshold < 0.0:
        raise ValueError('PID threshold has to be between 0.0 and 100.0')
    
    # load diamond results
    diamond_results = pd.read_csv(blasttsv, sep='\t', header=None)
    diamond_results.columns = ['query_ID', 'subject_ID', 'PID', 'align_len', 'no_mismatch', 'no_gapopen', 'query_start', 'query_end', 'subject_start', 'subject_end','E-value','bitscore']
    # filter by PID
    diamond_results = diamond_results[diamond_results['PID']>=pid_theshold]
    # trim cols
    diamond_results = diamond_results[['query_ID','subject_ID']]
    
    return diamond_results
    
dmnd_res = filter_DIAMOND_blastp_results('/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/DIAMOND_blastp_res.tsv', pid_theshold=40.0)

In [205]:
dmnd_res

,locus_tag,UniProt
0,SH0062,O05052


In [221]:
def map_dmnd_res_to_sp_ec_brenda(dmnd_results, swissprot_mapping_path):
    
    def _combine_EC_BRENDA(brenda,ec):
    
        nums_list = []
        if brenda and not pd.isna(brenda):
            for num in brenda.split(';'):
                nums_list.append(num.strip())

        if ec and not pd.isna(ec):
            for num in ec.split(';'):
                nums_list.append(num.strip())
            
        return nums_list
    
    # load the SwissProt mapping file
    swissprot_mapping = pd.read_csv(swissprot_mapping_path, sep='\t')
    swissprot_mapping.dropna(subset=['BRENDA','EC number'], how='all',inplace=True)
    
    # extract SwissProts IDs from subject_ID
    dmnd_results.columns = ['locus_tag','UniProt']
    dmnd_results['UniProt'] = dmnd_results['UniProt'].apply(lambda x: x.split('|')[1])
    
    # match
    dmnd_results = dmnd_results.merge(swissprot_mapping, left_on='UniProt', right_on='Entry', how='left')
    dmnd_results.drop('Entry', axis=1, inplace=True)
    dmnd_results['ec-code'] = dmnd_results.apply(lambda x: _combine_EC_BRENDA(x['BRENDA'],x['EC number']), axis=1)
    dmnd_results.drop(['BRENDA','EC number'], axis=1, inplace=True)
    dmnd_results = dmnd_results.explode('ec-code')
    
    return dmnd_results
    
    
map_dmnd_res_to_sp_ec_brenda(dmnd_res.copy(), '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/uniprot_table.tsv')

,locus_tag,UniProt,ec-code
0,SH0062,O05052,3.1.21.3
